In [1]:
import os
import requests
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

os.chdir("..")
load_dotenv()

True

In [2]:
def format_bulk_from_df(ticker: str, df: pd.DataFrame) -> bytes:
    """
    Build an NDJSON bulk payload for `ticker` from `df`.
    Expects DataFrame columns: 'open', 'close', 'high', 'low', 'volume' (adapt names as needed).
    """
    today = datetime.now().strftime('%Y-%m-%d')
    index_name = f"quant-agents_stocks-eod_{today}"
    lines = []

    for _, row in df.iterrows():

        date_reference = row.get('timestamp')
        open_ = row.get('open')
        close = row.get('close')
        high = row.get('high')
        low = row.get('low')
        volume = row.get('volume')

        if open_ is None or close is None:
            continue
        id_str = f"{ticker}_{str(date_reference)}"

        meta = {"index": {"_index": index_name, "_id": id_str}}

        doc = {
            "key_ticker": ticker,
            "date_reference": date_reference,
            "val_open": float(open_),
            "val_close": float(close),
            "val_high": float(high) if high is not None else None,
            "val_low": float(low) if low is not None else None,
            "val_volume": int(volume) if volume is not None else None,
        }

        lines.append(json.dumps(meta))
        lines.append(json.dumps(doc))

    return (("\n".join(lines)) + "\n").encode("utf-8")


In [4]:
es_url = os.environ.get('ELASTICSEARCH_URL')
es_api_key = os.environ.get('ELASTICSEARCH_API_KEY')
symbol = "IBM"
alpha_vantage_time_series_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey=demo&datatype=csv"

ibm_ticker_daily_time_series = pd.read_csv(alpha_vantage_time_series_url)

response = requests.post(
    url=f"{es_url}/_bulk",
    headers={
        'Authorization': f'ApiKey {es_api_key}',
        'Content-Type': 'application/x-ndjson'
    },
    data=format_bulk_from_df(symbol, ibm_ticker_daily_time_series)
)
print(response.json())

{'errors': False, 'took': 0, 'items': [{'index': {'_index': 'quant-agents_stocks-eod_2025-10-23', '_id': 'IBM_2025-10-22', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 100, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'quant-agents_stocks-eod_2025-10-23', '_id': 'IBM_2025-10-21', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 101, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'quant-agents_stocks-eod_2025-10-23', '_id': 'IBM_2025-10-20', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 102, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'quant-agents_stocks-eod_2025-10-23', '_id': 'IBM_2025-10-17', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 103, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'quant-agents_stocks-eod_202